# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pprint import pprint

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [34]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("../../data/train_data_0817.csv")
test_data = pd.read_csv("../../data/test_data_0817.csv")

In [35]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [36]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### train
var_all_train = [
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

In [37]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [38]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [39]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [40]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [41]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


---

## 모델링

### 모델 정의

In [69]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier(),
    'ada': AdaBoostClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

def stacking(train_data, test_data, base_models, final_model):
    """
    학습 데이터를 사용하여 모델을 학습하고, 테스트 데이터에 대해 예측 확률을 계산합니다.

    Parameters:
    train_data (pd.DataFrame): 학습 데이터 (특성 및 타겟 포함)
    test_data (pd.DataFrame): 테스트 데이터 (특성만 포함)
    base_models (list of tuples): 기본 모델 리스트 (이름, 모델 인스턴스)
    final_model (estimator): 최종 메타 모델

    Returns:
    np.array: 테스트 데이터에 대한 예측 확률
    StackingClassifier: 학습된 스태킹 모델
    """
    # 학습 데이터에서 특성과 타겟 분리
    X_train = train_data.drop(["target"], axis=1)
    y_train = train_data["target"].map({'Normal': 0, 'AbNormal': 1})

    # StackingClassifier 설정
    stacking_clf = StackingClassifier(
        estimators=base_models,
        final_estimator=final_model,
        passthrough=True  # 원본 특성도 메타 모델에 전달
    )

    # StackingClassifier 학습
    stacking_clf.fit(X_train, y_train)

    # 테스트 데이터에 대해 예측 확률 계산
    probabilities = stacking_clf.predict_proba(test_data)

    return probabilities, stacking_clf

### 모델 학습

Dam 모델

In [43]:
train_model_Dam = train_and_evaluate_model(
    'lgbm', train_data_dam
    , n_estimators=1467
    , num_leaves=2545
    , max_depth=37
    , learning_rate=0.04353920224587149 
    , min_child_samples=83
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.2222222222222222
---
Confusion Matrix:
[[7470  192]
 [ 361   79]]
---
Accuracy: 0.9317452480868921
Precision: 0.2915129151291513
Recall: 0.17954545454545454




AutoClave 모델

In [44]:
train_model_AutoClave = train_and_evaluate_model(
    'lgbm', train_data_autoclave
    , n_estimators=1563
    , num_leaves=1885
    , max_depth=15
    , learning_rate=0.07033655355880039 
    , min_child_samples=158
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.2511078286558346
---
Confusion Matrix:
[[7510  152]
 [ 355   85]]
---
Accuracy: 0.9374228585534435
Precision: 0.35864978902953587
Recall: 0.19318181818181818




Fill1 모델

In [45]:
train_model_Fill1 = train_and_evaluate_model(
    'lgbm', train_data_fill1
    , n_estimators=1452
    , num_leaves=1581
    , max_depth=22
    , learning_rate=0.002000452888170992 
    , min_child_samples=43
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.2260668973471742
---
Confusion Matrix:
[[7333  329]
 [ 342   98]]
---
Accuracy: 0.9171809429770427
Precision: 0.22950819672131148
Recall: 0.22272727272727272




Fill2 모델

In [46]:
train_model_Fill2 = train_and_evaluate_model(
    'lgbm', train_data_fill2
    , n_estimators=1632
    , num_leaves=1426
    , max_depth=8
    , learning_rate=0.07487990991624197 
    , min_child_samples=90
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.21999999999999997
---
Confusion Matrix:
[[7568   94]
 [ 374   66]]
---
Accuracy: 0.9422364848185634
Precision: 0.4125
Recall: 0.15




전체공정 모델

In [47]:
train_model_All = train_and_evaluate_model(
    'lgbm', train_data
    , n_estimators=2383
    , num_leaves=2528
    , max_depth=343
    , learning_rate=0.04661896043153508
    , min_child_samples=209
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data 데이터로 학습한 결과:
F1 Score: 0.24501424501424504
---
Confusion Matrix:
[[7486  176]
 [ 354   86]]
---
Accuracy: 0.9345840533201678
Precision: 0.3282442748091603
Recall: 0.19545454545454546




stacking

threshold 조절해보면서 f1-score 값이 높은것을 확인  
-> threshold를 선택하는 기준으로서 f1-score을 사용

In [48]:
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data.drop("target", axis=1),
    train_data["target"].map({'Normal': 0, 'AbNormal': 1}),
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# Base estimators 설정
estimators = [
    ('dam', train_model_Dam),
    ('autoclave', train_model_AutoClave),
    ('fill1', train_model_Fill1),
    ('fill2', train_model_Fill2)
]

# 최종 메타 모델 설정 (train_model_All 사용)
final_estimator = train_model_All

# StackingClassifier 설정
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    passthrough=True  # 원본 특성도 메타 모델에 전달
)

# StackingClassifier 학습
stacking_clf.fit(x_train, y_train)

# 예측 확률 계산
probabilities = stacking_clf.predict_proba(x_val)

# 임계값 적용 (예: 0.3)
threshold = 0.3
predictions = (probabilities[:, 1] >= threshold).astype(int)

# 평가
score = f1_score(y_val, predictions, average="binary")
conf_matrix = confusion_matrix(y_val, predictions)

print(f"Stacking Classifier F1 Score with threshold {threshold}: {score}")
print("Confusion Matrix:")
print(conf_matrix)

Stacking Classifier F1 Score with threshold 0.3: 0.19413919413919412


In [59]:
# 임계값 적용 
threshold = 0.3
predictions = (probabilities[:, 1] >= threshold).astype(int)

# 평가
score = f1_score(y_val, predictions, average="binary")
conf_matrix = confusion_matrix(y_val, predictions)

print(f"Stacking Classifier F1 Score with threshold {threshold}: {score}")
print("Confusion Matrix:")
print(conf_matrix)

Stacking Classifier F1 Score with threshold 0.3: 0.19413919413919412
Confusion Matrix:
[[7609   53]
 [ 387   53]]


In [61]:
# 임계값 적용 
threshold = 0.28
predictions = (probabilities[:, 1] >= threshold).astype(int)

# 평가
score = f1_score(y_val, predictions, average="binary")
conf_matrix = confusion_matrix(y_val, predictions)

print(f"Stacking Classifier F1 Score with threshold {threshold}: {score}")
print("Confusion Matrix:")
print(conf_matrix)

Stacking Classifier F1 Score with threshold 0.28: 0.19168173598553348
Confusion Matrix:
[[7602   60]
 [ 387   53]]


In [62]:
# 임계값 적용
threshold = 0.26
predictions = (probabilities[:, 1] >= threshold).astype(int)

# 평가
score = f1_score(y_val, predictions, average="binary")
conf_matrix = confusion_matrix(y_val, predictions)

print(f"Stacking Classifier F1 Score with threshold {threshold}: {score}")
print("Confusion Matrix:")
print(conf_matrix)

Stacking Classifier F1 Score with threshold 0.26: 0.19320214669051877
Confusion Matrix:
[[7597   65]
 [ 386   54]]


---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [63]:
model_Dam = fit_all_train_data_function(
    'lgbm', train_data_dam
    , n_estimators=1467
    , num_leaves=2545
    , max_depth=37
    , learning_rate=0.04353920224587149 
    , min_child_samples=83
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_AutoClave = fit_all_train_data_function(
    'lgbm', train_data_autoclave
    , n_estimators=1563
    , num_leaves=1885
    , max_depth=15
    , learning_rate=0.07033655355880039 
    , min_child_samples=158
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill1 = fit_all_train_data_function(
    'lgbm', train_data_fill1
    , n_estimators=1452
    , num_leaves=1581
    , max_depth=22
    , learning_rate=0.002000452888170992 
    , min_child_samples=43
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill2 = fit_all_train_data_function(
    'lgbm', train_data_fill2
    , n_estimators=1632
    , num_leaves=1426
    , max_depth=8
    , learning_rate=0.07487990991624197 
    , min_child_samples=90
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_All = fit_all_train_data_function(
    'lgbm', train_data
    , n_estimators=2383
    , num_leaves=2528
    , max_depth=343
    , learning_rate=0.04661896043153508
    , min_child_samples=209
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_dam 데이터로 학습 완료
lgbm 모델이 train_data_autoclave 데이터로 학습 완료
lgbm 모델이 train_data_fill1 데이터로 학습 완료
lgbm 모델이 train_data_fill2 데이터로 학습 완료
lgbm 모델이 train_data 데이터로 학습 완료


---

In [70]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

In [71]:
# Base estimators 설정
base_models = [
    ('dam', train_model_Dam),
    ('autoclave', train_model_AutoClave),
    ('fill1', train_model_Fill1),
    ('fill2', train_model_Fill2)
]
final_model = train_model_All

# 스태킹 기법을 사용하여 예측 확률 계산
probabilities, stacking_clf = stacking(train_data, x_test_all, base_models, final_model)

# 임계값 적용 (예: 0.3)
threshold = 0.3
final_predictions = (probabilities[:, 1] >= threshold).astype(int)

# 결과 출력
print(f'Final Predictions: {final_predictions}')
print("Sum of Final Predictions:", sum(final_predictions))

Final Predictions: [0 0 0 ... 0 0 0]
Sum of Final Predictions: 159


In [72]:
# 임계값을 조절
new_threshold = 0.3
new_predictions = (probabilities[:, 1] >= new_threshold).astype(int)
print(f'New Predictions with threshold {new_threshold}: {new_predictions}')
print("Sum of New Predictions:", sum(new_predictions))

New Predictions with threshold 0.3: [0 0 0 ... 0 0 0]
Sum of New Predictions: 159


In [73]:
# 임계값을 조절
new_threshold = 0.28
new_predictions = (probabilities[:, 1] >= new_threshold).astype(int)
print(f'New Predictions with threshold {new_threshold}: {new_predictions}')
print("Sum of New Predictions:", sum(new_predictions))

New Predictions with threshold 0.28: [0 0 0 ... 0 0 0]
Sum of New Predictions: 176


In [74]:
# 임계값을 조절
new_threshold = 0.26
new_predictions = (probabilities[:, 1] >= new_threshold).astype(int)
print(f'New Predictions with threshold {new_threshold}: {new_predictions}')
print("Sum of New Predictions:", sum(new_predictions))

New Predictions with threshold 0.26: [0 0 0 ... 0 0 0]
Sum of New Predictions: 192


## 4. 제출하기


### 제출 파일 작성


In [252]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("../data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

'제출하기' 이전의 스레스홀드를 통해 조절해서 나온 AbNormal 갯수와 동일한지 재확인

In [253]:
df_sub['target'].value_counts()

Normal      17234
AbNormal      127
Name: target, dtype: int64

In [254]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal


.